# Project IV: [OII] fluxes <a class="tocSkip">
    
the aim of this notebook is to combine the HII-region and cluster catalogues. 
    
In this notebook we do the matching on a per galaxie basis. For each resolution, a set of output files is produced that matches the nebulae to the association catalogue


In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from astrotools.packages import *

from astrotools.constants import tab10, single_column, two_column

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logger = logging.getLogger('pymuse')
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)-15s %(message)s",datefmt='%H:%M:%S')
handler.setFormatter(fmt)
logger.addHandler(handler)
basedir = Path('..')  # where we save stuff (and )
data_ext = Path('a:')/'Archive' # raw data

# we use the sample table for basic galaxy properties
sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

## Read in data

the galaxies listed in `hst_sample` have a cluster catalogue. The galaxies listed in `muse_sample` have astrosat observations to measure the FUV.

In [ ]:
# choose which version of the association catalogue to use
version = 'v1p2'
HSTband = 'nuv'
scalepc = 32

name = 'NGC2835'

### MUSE (DAP + nebulae catalogues)

In [ ]:
from pnlf.auxiliary import filter_table
from pnlf.io import ReadLineMaps

p = {x:sample_table.loc[name][x] for x in sample_table.columns}

# DAP linemaps (Halpha and OIII)
filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{name}*.fits'))
copt_res = float(filename.stem.split('-')[1].split('asec')[0])
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    Hbeta = NDData(data=hdul['HB4861_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HB4861_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HB4861_FLUX'].data),
                    meta=hdul['HB4861_FLUX'].header,
                    wcs=WCS(hdul['HB4861_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header)) 

# the original catalogue from Francesco
filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'Nebulae_catalogue_v2.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/'data_v2p1'/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)
nebulae = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])

nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])


nebulae['HIIregion'] = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)
HII_regions = filter_table(nebulae,gal_name=name,BPT_NII=0,BPT_SII=0,BPT_OI=0)
nebulae = filter_table(nebulae,gal_name=name)
nebulae.add_index('region_ID')
HII_regions.add_index('region_ID')

filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
    
print(f'{name}: {len(HII_regions)} HII-regions in final catalogue')

### HST

**white light + filter images**

In [ ]:
from cluster.io import read_associations

target  = name.lower()

# whitelight image (we set 0s to nan)
white_light_filename = data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits'
if white_light_filename.is_file():
    with fits.open(white_light_filename) as hdul:
        hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
        hst_whitelight.data[hst_whitelight.data==0] = np.nan
else:
    logging.warning('no white light image')
    
# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{name.lower()}_f275w_v1_exp-drc-sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))
filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{name.lower()}_f275w_v1_err-drc-wht.fits'
with fits.open(filename) as hdul:
    F275.uncertainty = StdDevUncertainty(hdul[0].data)
    
associations, associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',target=target,
                                                    HSTband=HSTband,scalepc=scalepc,version=version)


print(f'{name}: {len(associations)} associations in catalogue')    
# associations mask

### SITELLE [OII]

In [ ]:
from astrotools.regions import Regions
from reproject import reproject_interp

In [ ]:
with fits.open(basedir/'data'/'data_v2p1'/'maps'/'NGC2835_OII_map_reprojected.fits') as hdul:
    OII = NDData(data=hdul[1].data,
                 meta=hdul[1].header,
                 wcs=WCS(hdul[1].header))

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,4))

for name,ax in zip(['NGC0628','NGC2835','NGC3351'],[ax1,ax2,ax3]):
    with fits.open(basedir/'data'/'data_v2p1'/'maps'/f'{name}_OII_map.fits') as hdul:
        OII_img = hdul['OII3726_FLUX'].data
        OII_header = hdul['OII3726_FLUX'].header
    
    norm = simple_norm(OII_img,clip=False,percent=98)
    ax.imshow(OII_img,norm=norm,origin='lower',cmap=plt.cm.Greys)
    ax.set_title(name)
    ax.axis('off')
    
plt.show()   

## Compare Strong line prescriptions

use [OII] line to calculate strong line and direct abundances

In [ ]:
out_folder = 'data_v2p2'


from astrotools.metallicity import diagnostic_line_ratios

with fits.open(basedir / 'data' / out_folder / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/out_folder/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)

nebulae_with_OII = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
nebulae_with_OII = nebulae_with_OII[np.isin(nebulae_with_OII['gal_name'],['NGC0628','NGC2835','NGC3351','NGC4535'])]
nebulae_with_OII = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>0]

line_ratios = diagnostic_line_ratios(nebulae_with_OII)

compare with Figure 8 in Pilyugin+2016 (looks good if R2*=1.4)

In [ ]:
from astrotools.metallicity import strong_line_metallicity_R, strong_line_metallicity_S

subsample = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX']>1*nebulae_with_OII['OII3726_FLUX_ERR']].copy()
print(f'{len(subsample)} objects in sample')

# looks a lot better with 1.4*R2
subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_S'],subsample['OH_R'],s=4,c=tab10[0])

ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H)$_\mathrm{S}$',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_S']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H)$_\mathrm{S}-$log(O/H)$_\mathrm{R}$')
plt.savefig(basedir/'reports'/'12+logOH R vs S calibration.pdf',dpi=300)
plt.show()

with direct method. This requires electron temperature and density. They have to be measured in an itterative process

In [ ]:
from astrotools.metallicity import electron_density_sulfur,\
                                electron_temperature_oxygen, electron_temperature_nitrogen,\
                                electron_temperature_sulfur, oxygen_abundance_direct
   
criteria = (line_ratios['OII7319_FLUX_CORR']>7*line_ratios['OII7319_FLUX_CORR_ERR']) & (line_ratios['OII3726_FLUX_CORR']>10*line_ratios['OII3726_FLUX_CORR_ERR'])
subsample = line_ratios[criteria].copy()

subsample['OH_R'] = strong_line_metallicity_R(subsample['R2'],subsample['R3'],subsample['N2'])
subsample['OH_S'] = strong_line_metallicity_S(subsample['S2'],subsample['R3'],subsample['N2'])
    
# initial guess for the temperature
subsample['t(NII)'] = electron_temperature_nitrogen(subsample['RN2'])
subsample['t(SIII)'] = electron_temperature_sulfur(subsample['RS3'])
subsample['n(SII)']  = electron_density_sulfur(subsample['RS2'],subsample['t(NII)'])

for x in range(10):
    subsample['t(OII)'] = electron_temperature_oxygen(subsample['RO2'],subsample['n(SII)'])
    subsample['n(SII)'] = electron_density_sulfur(subsample['RS2'],subsample['t(OII)'])
    print(np.nanmean(subsample['n(SII)']))

subsample['OH_direct'] = oxygen_abundance_direct(subsample['R2'],subsample['R3'],subsample['t(OII)'],subsample['n(SII)'])


In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(6,3))

ax1.plot([8.1,8.8],[8.1,8.8],color='black')
ax1.plot([8.1,8.8],[8.2,8.9],color='grey',ls='--')
ax1.plot([8.1,8.8],[8.0,8.7],color='grey',ls='--')
ax1.scatter(subsample['OH_direct'],subsample['OH_R'],s=4,c=tab10[0])
ax1.set(xlim=[8.1,8.8],ylim=[8.1,8.8],
       xlabel='12+log(O/H) direct',
       ylabel='12+log(O/H)$_\mathrm{R}$')
ax2.hist(subsample['OH_direct']-subsample['OH_R'],bins=np.linspace(-0.3,0.3,20),histtype='step',color='black')
ax2.set(xlabel=r'log(O/H) direct$-$log(O/H)$_\mathrm{R}$')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

compare with Figure 7 in Perez-Montero+2017

In [ ]:
fig,ax=plt.subplots(figsize=(6,4))

ax.scatter(np.log10(subsample['R23']),subsample['OH_direct'],s=4,c=subsample['logq_D91'])
ax.set(xlim=[-0.4,1.4],ylim=[7.,9.0],
       xlabel='log R23',
       ylabel='12+log(O/H) direct')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

ax.plot([0.5,1.5],[0.5,1.5],color='black')
ax.scatter(subsample['t(OII)'],subsample['t(NII)'],s=4,c=tab10[0])
ax.set(xlim=[0.5,1.5],ylim=[0.5,1.5],
       xlabel='t([OII])',
       ylabel='t([NII])')
#plt.savefig('12+logOH R vs S calibration.png',dpi=600)
plt.show()

## Compare ionisation parameter

see also Kewley and Dopita 2002 (Fig 17)

In [ ]:
out_folder = 'data_v2p2'


with fits.open(basedir / 'data' / out_folder / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/out_folder/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)

nebulae = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
nebulae = nebulae[np.isin(nebulae['gal_name'],['NGC0628','NGC2835','NGC3351'])]

with np.errstate(divide='ignore',invalid='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    #SIII = nebulae['SIII6312_FLUX_CORR']+nebulae['SIII9068_FLUX_CORR']
    SIII = (1+2.47)*nebulae['SIII9068_FLUX_CORR']
    nebulae['[SIII]/[SII]'][SII>0] = SIII[SII>0]/SII[SII>0]
    
    nebulae['[OIII]/[OII]'] = (1+0.35) * nebulae['OIII5006_FLUX_CORR'] / ((1+0.714)*nebulae['OII3726_FLUX_CORR'])
    

In [ ]:
Ionization_Parameter_Coefficients = {
    'OII' : {0.05:(-36.9772, 10.2838, -0.957421, 0.0328614),
             0.1:(-74.2814, 24.6206, -2.79194, 0.110773),
             0.2:(-36.7948, 10.0581, -0.914212, 0.0300472),
             0.5:(81.188, 27.5082, -3.19126, 0.128252),
             1.0:(-52.6367, 16.088, -1.67443, 0.0608004),
             1.5:(-86.8674, 28.0455, -3.01747, 0.108311),
             2.0:(-24.4044, 2.51913, 0.452486, -0.0491711),
             3.0:(49.4728, -27.4711, 4.50304, -0.232228)
            },
    
    'OII2' : {0.05:(7.39167, 0.667891, 0.0680367),
             0.1:(7.46218, 0.685835, 0.0866086),
             0.2:(7.57817, 0.739315, 0.084364),
             0.5:(7.73013, 0.843125, 0.118166)
            },
    
    'SII' : {0.05:(30.0116, -14.897, 2.30577, -0.112314),
             0.1:(16.8569, -9.62876, 1.59938, -0.0804552),
             0.2:(32.2358, -15.2438, 2.27251, -0.106913),
             0.5:(-3.06247, -0.864092, 0.328467, -0.0196089),
             1.0:(-2.94394, -0.546041, 0.239226, -0.0136716),
             1.5:(-38.1338, 13.0914, -1.51014, 0.0605926),
             2.0:(-21.024, 6.55748, -0.683584, 0.025869),
             3.0:(-6.61131, 1.36836, -0.071756, 0.00225792)
            },
}


In [ ]:
from scipy.stats import spearmanr

tmp = nebulae[nebulae['OII3726_FLUX_CORR']>1*nebulae['OII3726_FLUX_CORR_ERR']]

fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

cmap = mpl.cm.get_cmap('viridis')
norm = mpl.colors.Normalize(vmin=0.05,vmax=3)


sc=ax.scatter(10**(tmp['logq_D91']),np.log10(tmp['[OIII]/[OII]']),c=tmp['met_scal'],vmin=8.25,vmax=8.65)

q = np.logspace(6.,8.5)
for z,coefficients in Ionization_Parameter_Coefficients['OII'].items():
    R = np.polyval(coefficients[::-1],np.log10(q))
    ax.plot(q,R,label=z,color=cmap(norm(z)))
    
ax.legend(ncol=3)
ax.set(xlim=[1e6,2e8],xscale='log',ylim=[-2.5,2],xlabel=r'$q$',ylabel='log [OIII]/[OII]')

fig.colorbar(sc,label=r'$12+\log(\mathrm{O}/\mathrm{H})$')

plt.show()

## MUSE vs SITELLE velocity

In [ ]:
out_folder = 'data_v2p2'

with fits.open(basedir / 'data' / out_folder / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/ out_folder /'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)

nebulae_with_OII = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
nebulae_with_OII = nebulae_with_OII[np.isin(nebulae_with_OII['gal_name'],['NGC0628','NGC2835','NGC3351','NGC4535'])]

with fits.open(basedir/'data'/'phangs_sample_table_v1p6.fits') as hdul:
    sample_table = Table(hdul[1].data)
sample_table['name'] = [x.upper() for x in sample_table['name']]
sample_table.add_index('name')

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

table = nebulae_with_OII[nebulae_with_OII['OII3726_FLUX_CORR']>1*nebulae_with_OII['OII3726_FLUX_CORR_ERR']]

lim = [400,1200]
for gal_name in ['NGC0628','NGC2835','NGC3351']:
    tmp = table[table['gal_name']==gal_name]
    print(f'{gal_name}: {len(tmp)} objects')
    velocity = sample_table.loc[gal_name]['orient_vlsr']
    ax.scatter(velocity+tmp['NII6583_VEL'],tmp['OII3726_velocity'],alpha=0.5,label=gal_name)
    
ax.legend()
offset = 90
ax.plot(lim,np.array(lim)+offset,color='black')
ax.plot(lim,np.array(lim)+offset+30,color='gray')
ax.plot(lim,np.array(lim)+offset-30,color='gray')
ax.set(xlim=lim,ylim=lim,xlabel='[NII] velocity',ylabel='[OII] velocity')

plt.show()

## Check if we really are in the low density limit

$$
\frac{[SII]6717}{[SII]6731} = 1.49 \frac{1+3.77x}{1+12.8x} 
$$
with $x=10^{-4} n_e t^{-1/2}$ with $t$ in units of $10^{4}$ K.

(see equation 1 in https://doi.org/10.1093/mnras/stac456)

or Sanders+2016 (https://iopscience.iop.org/article/10.3847/0004-637X/816/1/23/pdf)

In [ ]:
from astrotools.metallicity import ratio_from_electron_density

with fits.open(basedir / 'data' / 'data_v2p1' / 'Nebulae_Catalogue_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(6, 3),
                       gridspec_kw={
                           'width_ratios': [2.5, 1],
                           'wspace': 0.},sharey=True)

n_e = np.logspace(1,5,200)
ax1.plot(n_e,ratio_from_electron_density(n_e,'SII'),label='[SII]')
ax1.plot(n_e,ratio_from_electron_density(n_e,'OII'),label='[OII]')
ax1.plot([10,1e5],[1.4,1.4],color='gray')

ax2.hist(nebulae['SII6716_FLUX_CORR']/nebulae['SII6730_FLUX_CORR'],bins=np.arange(1,1.7,0.05),
         orientation='horizontal')
ax1.set(xscale='log',xlabel='Electron density / cm$^{-3}$',xlim=[10,1e5],
        ylabel='ratio')
ax1.legend()

plt.show()

do the same thing for the TYPHOON data

In [ ]:
typhoon_catalogue = ascii.read(basedir/'data'/'typhoon'/'NGC2835_HIIdata_notdereddened_WCS.dat',names=names)

fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                       gridspec_kw={
                           'width_ratios': [2.5, 1],
                           'wspace': 0.},sharey=True)

n_e = np.logspace(1,5,200)
ax1.plot(n_e,ratio_from_electron_density(n_e,'SII'))
ax1.plot([10,1e5],[1.4,1.4],color='gray')
ax1.set(xlim=[10,1e5])
ax2.hist(typhoon_catalogue['SII6716']/typhoon_catalogue['SII6731'],bins=np.arange(1,2,0.05),
         orientation='horizontal')
ax1.set(xscale='log',xlabel='Electron density / cm$^{-3}$')

plt.show()

## Compare fitted spectrum to line map



In [ ]:
nebulae_mask_OII = reproject_interp(nebulae_mask,
                                     output_projection=Halpha.wcs,
                                     shape_out=OII.data.shape,
                                     order='nearest-neighbor',return_footprint=False)  

In [ ]:
# the shape of the [OII] map is wrong
nebulae['OII3726_FLUX_NEW'] = np.array([np.nansum(OII.data[nebulae_mask_OII==region_ID]) for region_ID in nebulae['region_ID']])*1e20 
nebulae['HA6562_FLUX_NEW'] = np.array([np.nansum(Halpha.data[nebulae_mask.data==region_ID]) for region_ID in nebulae['region_ID']]) 

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e2,1e6]
ax.scatter(nebulae['OII3726_FLUX'],nebulae['OII3726_FLUX_NEW']/4,s=1)
ax.plot(lim,lim,color='black')
ax.set(xlabel=r'[O\,\textsc{ii}] (fitted spectrum)',ylabel=r'[O\,\textsc{ii}] (line map)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(nebulae['OII3726_FLUX'],nebulae['OII3726_FLUX_NEW'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
ax.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

plt.show()

## TYPHOON

we can not use the MUSE priors for the TYPHOON masks. Hence this is based on the old version

In [ ]:
with fits.open(basedir/'data'/'typhoon'/'NGC2835_Halpha_WCS.fits') as hdul:
    typhoon_halpha = NDData(data=hdul[0].data,
                 meta=hdul[0].header,
                 wcs=WCS(hdul[0].header))
with fits.open(basedir/'..'/'sitelle'/'data'/'typhoon'/'N2835_TG10_sig3.50000_hiiphot_WCS.fits') as hdul:
    typhoon_mask = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=typhoon_halpha.wcs)

names = ['HIIregion','Rgal(kpc)','Halpha','Halpha_e','Hbeta','Hbeta_e',
         'OIII5007','OIII5007_e','OIII4636','OIII4636_e',
         'NII6583','NII6583_e','SII6716,31','SII6716,31_e',
         'OII3726,29','OII3726,29_e','SII6716','SII6716_e',
         'SII6731','SII6731_e']

typhoon_catalogue = ascii.read(basedir/'data'/'typhoon'/'NGC2835_HIIdata_notdereddened_WCS.dat',names=names)
typhoon_catalogue.rename_column('HIIregion','region_ID')

with fits.open(basedir/'data'/'typhoon'/'NGC2835_typhoon_OII_fluxes.fits') as hdul:
    OII_typhoon = Table(hdul[1].data)
typhoon_catalogue = join(typhoon_catalogue,OII_typhoon,keys='region_ID')

import pyneb as pn
EBV_NGC2835 = 0.089

rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_NGC2835,law='CCM89 oD94')

columns_wavelength = {'Halpha':6562,'Halpha_e':6562,'Hbeta':4861,'Hbeta_e':4861,
                      'OIII5007':5006,'OIII5007_e':5007,'OIII4636':4636,'OIII4636_e':4636,
                      'NII6583':6583,'NII6583_e':6583,'SII6716,31':6716,'SII6716,31_e':6716,
                      'OII3726,29':3726,'OII3726,29_e':3726,'SII6716':6716,'SII6716_e':6716,
                      'SII6731':6731,'SII6731_e':6731}

# the TYPHOON fluxes are not extinction corrected (MW)
for col, wav in columns_wavelength.items():
    typhoon_catalogue[col] *= 1e3 # * rc_MW.getCorr(wav)

the TYPHOON H$\alpha$ with the regions overplotted

### use the reprojected map to measure the fluxes

In [ ]:
from astrotools.regions import Regions
from reproject import reproject_interp

typhoon_regions = Regions(typhoon_mask.data,projection=typhoon_mask.wcs)

typhoon_muse, footprint = reproject_interp(typhoon_mask,
                                 output_projection=nebulae_mask.wcs,
                                 shape_out=nebulae_mask.data.shape,
                                 order='nearest-neighbor',return_footprint=True)    
typhoon_regions_muse = Regions(typhoon_muse,projection=Halpha.wcs)

# show the footprint of the MUSE Halpha
_, footprint = reproject_interp(Halpha,
                                output_projection=typhoon_mask.wcs,
                                shape_out=typhoon_mask.data.shape,
                                order='nearest-neighbor',return_footprint=True) 

typhoon_catalogue['outside'] = [np.any(Halpha.mask[typhoon_muse==region_ID]) for region_ID in typhoon_catalogue['region_ID']]
typhoon_catalogue['size'] = [np.sum(typhoon_muse==region_ID) for region_ID in typhoon_catalogue['region_ID']]

look at the reprojected TYPHOON masks

In [ ]:
fig = plt.figure(figsize=(1.5*two_column,1.3*two_column/3))

ax1 = fig.add_subplot(131,projection=Halpha.wcs)
norm = simple_norm(typhoon_halpha.data,clip=False,percent=99.6)
ax1.imshow(typhoon_halpha.data,norm=norm,cmap=plt.cm.gray_r)
ax1.imshow(footprint,alpha=0.4,cmap=plt.cm.Blues)
for cont in typhoon_regions.contours:
    ax1.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax1.set(xlabel='R.A.',ylabel='Dec.',ylim=[250,450])
ax1.set_title(r'TYPHOON H$\alpha$',fontsize=7)

ax2 = fig.add_subplot(132,projection=Halpha.wcs)
norm = simple_norm(Halpha.data,clip=False,percent=99.6)
ax2.imshow(Halpha.data,norm=norm,cmap=plt.cm.gray_r)
for cont in typhoon_regions_muse.contours:
    ax2.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax2.set(xlabel='R.A.',ylabel=' ')
#ax2.coords[1].set_ticklabel_visible(False)
ax2.set_title(r'MUSE H$\alpha$',fontsize=7)


ax3 = fig.add_subplot(133,projection=OII.wcs)
norm = simple_norm(OII.data,clip=False,percent=99.6)
ax3.imshow(OII.data,norm=norm,cmap=plt.cm.gray_r)
for cont in typhoon_regions_muse.contours:
    ax3.plot(cont[:,1], cont[:,0], 'k-',lw=0.2,color='red')
ax3.set(xlabel='R.A.',ylabel='Dec.')
ax3.coords[1].set_ticklabel_visible(False)
ax3.set_title('SITELLE [OII]',fontsize=7)

plt.tight_layout()
#plt.savefig(basedir/'reports'/'TYPHOON_regions_reprojected.pdf',dpi=300)
plt.show()

In [ ]:
tmp = typhoon_catalogue.copy()
tmp = tmp[np.isin(tmp['region_ID'],typhoon_muse)]

tmp = tmp[~tmp['outside']]
tmp = tmp[tmp['SII6716']/tmp['SII6731']>1.4]

# the fluxes in typhon are in units of 1e-17 erg / cm2 / s
tmp['Halpha_muse'] = np.array([np.nansum(Halpha.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['Hbeta_muse'] = np.array([np.nansum(Hbeta.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['OIII_muse'] = np.array([np.nansum(OIII.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) 
tmp['OII_sitelle'] = np.array([np.nansum(OII.data[typhoon_muse==region_ID]) for region_ID in tmp['region_ID']]) * 1e20 

# we need both lines [OII]3729 = 0.35 * [OII]3726 and extinction correction
tmp['OII_sitelle'] *= rc_MW.getCorr(3726) * 1.35
TYPHOON_to_MUSE = (OII.wcs.pixel_scale_matrix[0][0] / typhoon_halpha.wcs.pixel_scale_matrix[0][0])**2


tmp['OII_sitelle_fit'] = tmp['OII3726_FLUX']+tmp['OII3729_FLUX']
#rc_balmer = pn.RedCorr(R_V=3.1,law='CCM89 oD94')
#rc_balmer.setCorr(obs_over_theo=tmp['Halpha_muse']/tmp['Hbeta_muse'] / 2.86,wave1=6562.81,wave2=4861.33)

print(f'{len(tmp)} objects in catalogue')

In [ ]:
from scipy.stats import spearmanr

fig,((ax1,ax2,ax3),(ax4,ax5,ax6))=plt.subplots(ncols=3,nrows=2,figsize=(two_column,two_column/1.5))
factor = 1
kwargs = {'c' : tmp['size'], 'vmin':0,'vmax':4000}
kwargs = {}

lim = [1e5,2e7]
ax1.scatter(tmp['Halpha'],tmp['Halpha_muse'],s=1,**kwargs)
ax1.plot(lim,lim,color='black')
ax1.set(xlabel=r'H$\alpha$ (TYPHOON)',ylabel=r'H$\alpha$ (MUSE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['Halpha'],tmp['Halpha_muse'],nan_policy='omit')
print(f'Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax1.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [1e5,2e7]
ax2.scatter(tmp['Hbeta'],tmp['Hbeta_muse'],s=1,**kwargs)
ax2.plot(lim,lim,color='black')
ax2.set(xlabel=r'H$\beta$ (TYPHOON)',ylabel=r'H$\beta$ (MUSE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['Hbeta'],tmp['Hbeta_muse'],nan_policy='omit')
print(f'Hbeta: rho={rho:.2f}, p-value={p:.2g}')
#ax2.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [1e5,2e7]
ax3.scatter(tmp['OII3726,29'],tmp['OII_sitelle_fit']/factor,s=1,**kwargs)
ax3.plot(lim,lim,color='black')
ax3.set(xlabel=r'[O\,\textsc{ii}] (TYPHOON)',ylabel=r'[O\,\textsc{ii}] (SITELLE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['OII3726,29'],tmp['OII_sitelle_fit'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
#ax3.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim = [0,1]
ax4.scatter(tmp['OIII5007']/tmp['Halpha'],tmp['OIII_muse']/tmp['Halpha_muse'],s=1,**kwargs)
ax4.plot(lim,lim,color='black')
ax4.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{iii}]/H$\alpha$ (TYPHOON)',ylabel=r'[O\,\textsc{iii}]/H$\alpha$ (MUSE)')
rho,p=spearmanr(tmp['OIII5007']/tmp['Halpha'],tmp['OIII_muse']/tmp['Halpha_muse'],nan_policy='omit')
print(f'OIII/Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax4.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim=[0,2]
ax5.scatter(tmp['OII3726,29']/tmp['Halpha'],tmp['OII_sitelle_fit']/tmp['Halpha_muse']/factor,s=1,**kwargs)
ax5.plot(lim,lim,color='black')
ax5.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{ii}]/H$\alpha$ (TYPHOON)',ylabel=r'[O\,\textsc{ii}]/H$\alpha$ (SITELLE/MUSE)')
rho,p=spearmanr(tmp['OII3726,29']/tmp['Halpha'],tmp['OII_sitelle_fit']/tmp['Halpha_muse'],nan_policy='omit')
print(f'OII/Halpha: rho={rho:.2f}, p-value={p:.2g}')
#ax5.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

lim=[0,4.5]
ax6.scatter(tmp['OII3726,29']/tmp['Hbeta'],tmp['OII_sitelle_fit']/tmp['Hbeta_muse']/factor,s=1,**kwargs)
ax6.plot(lim,lim,color='black')
ax6.set(xlim=lim,ylim=lim,xlabel=r'[O\,\textsc{ii}]/H$\beta$ (TYPHOON)',ylabel=r'[O\,\textsc{ii}]/H$\beta$ (SITELLE/MUSE)')
rho,p=spearmanr(tmp['OII3726,29']/tmp['Hbeta'],tmp['OII_sitelle_fit']/tmp['Hbeta_muse'],nan_policy='omit')
print(f'OII/Hbeta: rho={rho:.2f}, p-value={p:.2g}')
#ax6.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)

plt.tight_layout()
#plt.savefig(basedir/'reports'/'TYPHOON_flux_comparison.pdf',dpi=300)
plt.show()

In [ ]:
from scipy.stats import spearmanr

fig,ax=plt.subplots(figsize=(single_column,single_column))

lim = [1e5,5e7]
ax.scatter(tmp['OII_sitelle_fit'],tmp['OII_sitelle'],s=1)
ax.plot(lim,lim,color='black')
ax.set(xlabel=r'[O\,\textsc{ii}] (integrated spectra)',ylabel=r'[O\,\textsc{ii}] (map)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)
rho,p=spearmanr(tmp['OII3726,29'],tmp['OII_sitelle'],nan_policy='omit')
print(f'[OII]: rho={rho:.2f}, p-value={p:.2g}')
ax.set_title(r'$\rho=$'+f'{rho:.2f}, p-value={p:.2g}',fontsize=8)
#plt.savefig(basedir/'reports'/'integrated_spectra_vs_map.png',dpi=300)

plt.show()

### Also compare with KCWI

In [ ]:
from reproject import reproject_interp

with fits.open(basedir/'data'/'KCWI'/f'{name.lower()}_OII_map.fits') as hdul:
    OII_KCWI = NDData(data=hdul[1].data,
                 meta=hdul[1].header,
                 wcs=WCS(hdul[1].header))
OII_KCWI.data[OII_KCWI.data==0]=np.nan

typhoon_kcwi, footprint = reproject_interp(typhoon_mask,
                                 output_projection=OII_KCWI.wcs,
                                 shape_out=OII_KCWI.data.shape,
                                 order='nearest-neighbor',return_footprint=True)

In [ ]:
import pyneb as pn

tmp = typhoon_catalogue.copy()
tmp['OII_sitelle_fit'] = tmp['OII3726_FLUX']+tmp['OII3729_FLUX']

EBV_NGC2835 = 0.089
EBV_NGC0628 = 0.062

rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_NGC2835,law='CCM89 oD94')
TYPHOON_to_KCWI = (OII_KCWI.wcs.pixel_scale_matrix[0][0] / typhoon_halpha.wcs.pixel_scale_matrix[0][0])**2

tmp['OII_KCWI'] = np.array([np.sum(OII_KCWI.data[typhoon_kcwi==region_ID]) for region_ID in tmp['region_ID']]) 
# the fluxes in KCWI are in units of 1e-16 and we use 1e-20
tmp['OII_KCWI'] *= 1e4*rc_MW.getCorr(3726) / TYPHOON_to_KCWI


In [ ]:
fig,ax=plt.subplots()

norm = simple_norm(OII_KCWI.data,clip=False,percent=99)
ax.imshow(OII_KCWI.data,norm=norm,cmap=plt.cm.Greys,origin='lower')
ax.imshow(typhoon_kcwi,alpha=0.3,origin='lower')
ax.set_title('TYPHOON masks over [OII] KCWI')
ax.axis('off')

plt.show()

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3))

lim = [1e5,2e7]

ax1.scatter(tmp['OII3726,29'],tmp['OII_sitelle_fit'],s=1)
ax1.plot(lim,lim,color='black')
ax1.set(xlabel=r'[O\,\textsc{ii}] (TYPHOON)',ylabel=r'[O\,\textsc{ii}] (SITELLE)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)

ax2.scatter(tmp['OII_sitelle_fit'],tmp['OII_KCWI'],s=1)
ax2.plot(lim,lim,color='black')
ax2.set(xlabel=r'[O\,\textsc{ii}] (SITELLE)',ylabel=r'[O\,\textsc{ii}] (KCWI)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)

ax3.scatter(tmp['OII_KCWI'],tmp['OII3726,29'],s=1)
ax3.plot(lim,lim,color='black')
ax3.set(xlabel=r'[O\,\textsc{ii}] (KCWI)',ylabel=r'[O\,\textsc{ii}] (TYPHOON)',
        xscale='log',yscale='log',xlim=lim,ylim=lim)

plt.tight_layout()

plt.show()

### use the reprojected regions to measure the flux

In [ ]:
typhoon_regions = Regions(typhoon_mask.data,projection=typhoon_mask.wcs)
typhoon_regions_muse = typhoon_regions.reproject(Halpha.wcs,shape=Halpha.data.shape)

## KCWI

In [ ]:
from reproject import reproject_exact
from astrotools.regions import Regions

name = 'NGC2835'

with fits.open(basedir/'data'/'data_v2p1'/'maps'/f'{name}_OII_map_reprojected.fits') as hdul:
    OII_SITELLE = NDData(data=hdul[1].data,
                 meta=hdul[1].header,
                 wcs=WCS(hdul[1].header))
    
with fits.open(basedir/'data'/'KCWI'/f'{name.lower()}_OII_map.fits') as hdul:
    OII_KCWI = NDData(data=hdul[1].data,
                 meta=hdul[1].header,
                 wcs=WCS(hdul[1].header))
OII_KCWI.data[OII_KCWI.data==0]=np.nan

# DAP linemaps (Halpha and OIII)
filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{name}*.fits'))
copt_res = float(filename.stem.split('-')[1].split('asec')[0])
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    
filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# the original catalogue from Francesco
filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'Nebulae_catalogue_v2.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
with fits.open(basedir/'data'/'data_v2p2'/'fluxes'/'NGC2835_OII_fluxes.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)
nebulae = join(nebulae,OII_fluxes,keys=['gal_name','region_ID'])
    
# reproject the nebulae masks
from reproject import reproject_interp
nebulae_mask_KCWI, footprint = reproject_interp(nebulae_mask,
                                 output_projection=OII_KCWI.wcs,
                                 shape_out=OII_KCWI.data.shape,
                                 order='nearest-neighbor',return_footprint=True)  
overlap = np.unique(nebulae_mask_KCWI[~np.isnan(OII_KCWI.data)])


muse_regions = Regions(nebulae_mask.data,projection=nebulae_mask.wcs)

OII_KCWI_reprojected, footprint = reproject_exact(OII_KCWI,OII_SITELLE.meta)
xvalues = np.where(np.any(footprint,axis=0))
xmin,xmax = np.min(xvalues),np.max(xvalues)
yvalues = np.where(np.any(footprint,axis=1))
ymin,ymax = np.min(yvalues),np.max(yvalues)

### Look at the linemaps

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(8,3))

norm=simple_norm(OII_SITELLE.data,clip=False,percent=99)

norm=simple_norm(OII_SITELLE.data,clip=False,percent=99)
ax1.imshow(OII_SITELLE.data,norm=norm,origin='lower',cmap=plt.cm.gray_r)
ax1.imshow(~np.isnan(OII_KCWI_reprojected),origin='lower',alpha=0.4,cmap=plt.cm.Blues)
ax1.set_title(r'[O\,\textsc{ii}] (SITELLE)')
ax1.axis('off')

ax2.imshow(OII_SITELLE.data,norm=norm,origin='lower',cmap=plt.cm.gray_r)
ax2.set_title(r'[O\,\textsc{ii}] (SITELLE)')
ax2.axis('off')

norm=simple_norm(OII_KCWI_reprojected.data,clip=False,percent=99)
ax3.imshow(OII_KCWI_reprojected.data,norm=norm,origin='lower',cmap=plt.cm.gray_r)
ax3.axis('off')

for cont,label in zip(muse_regions.contours,muse_regions.labels):
    ax3.plot(cont[:,1], cont[:,0], '-',lw=0.5,color='red')

ax3.set_title(r'[O\,\textsc{ii}] (KCWI)')

'''
img = OII_SITELLE.data / (OII_KCWI_reprojected.data) *1e16
#norm=simple_norm(img,clip=False,percent=99,stretch='log',min_cut=-0.3,max_cut=0.3)
im = ax3.imshow(np.log10(img),vmin=-0.3,vmax=2.5,origin='lower',cmap=plt.cm.gray_r)
fig.colorbar(im)
'''
ax2.set(xlim=[xmin,xmax],ylim=[ymin,ymax])
ax3.set(xlim=[xmin,xmax],ylim=[ymin,ymax])

plt.savefig(basedir/'reports'/'KCWI_regions_reprojected.pdf',dpi=300)


plt.show()

### Measure flux in HII regions from nebulae catalogue

reproject masks instead of line map

In [ ]:
import pyneb as pn
EBV_NGC2835 = 0.089
EBV_NGC0628 = 0.062

rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_NGC2835,law='CCM89 oD94')
MUSE_to_KCWI = (OII_KCWI.wcs.pixel_scale_matrix[0][0] / Halpha.wcs.pixel_scale_matrix[0][0])**2

with fits.open(basedir/'data'/'data_v2p2'/'fluxes'/'NGC2835_OII_fluxes.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)
OII_fluxes = OII_fluxes[OII_fluxes['gal_name']==name]
OII_fluxes = OII_fluxes[np.isin(OII_fluxes['region_ID'],overlap)]
OII_fluxes['OII_KCWI'] = np.array([np.sum(OII_KCWI.data[nebulae_mask_KCWI==region_ID]) for region_ID in OII_fluxes['region_ID']]) 
# the fluxes in KCWI are in units of 1e-16 and we use 1e-20
OII_fluxes['OII_KCWI'] *= 1e4*rc_MW.getCorr(3726) #* MUSE_to_KCWI

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

lim = [1e3,1e6]
tmp = OII_fluxes[OII_fluxes['OII3726_FLUX']<1*OII_fluxes['OII3726_FLUX_ERR']]
print(len(tmp))
ax.scatter(tmp['OII_KCWI'],tmp['OII3726_FLUX'],color='0.8')

tmp = OII_fluxes[OII_fluxes['OII3726_FLUX']>=1*OII_fluxes['OII3726_FLUX_ERR']]
print(len(tmp))
ax.scatter(tmp['OII_KCWI'],tmp['OII3726_FLUX'])

ax.plot(lim,lim,color='black')
ax.set(xscale='log',xlim=lim,yscale='log',ylim=lim,
       ylabel=r'[O\,\textsc{ii}] (SITELLE)',xlabel=r'[O\,\textsc{ii}] (KCWI)')
plt.savefig(basedir/'reports'/'KCWI_flux_comparison.pdf',dpi=300)

plt.show()

which regions have the biggest difference

In [ ]:
tmp = OII_fluxes[OII_fluxes['OII3726_FLUX']>=1*OII_fluxes['OII3726_FLUX_ERR']]
mask = np.full_like(nebulae_mask_KCWI,np.nan)
bad = np.full_like(nebulae_mask_KCWI,np.nan)

for row in tmp:
    diff = 100*np.abs((row['OII_KCWI']-row['OII3726_FLUX'])/row['OII_KCWI'])
    mask[nebulae_mask_KCWI==row['region_ID']]=diff
    if np.isnan(diff):
        bad[nebulae_mask_KCWI==row['region_ID']]=1

In [ ]:
fig = plt.figure(figsize=(two_column,two_column/2))



ax1 = fig.add_subplot(121,projection=OII_KCWI.wcs)
norm=simple_norm(OII_KCWI.data,clip=False,percent=99)
ax1.imshow(OII_KCWI.data,norm=norm,cmap=plt.cm.gray_r)

tmp = OII_fluxes[OII_fluxes['OII3726_FLUX']>=1*OII_fluxes['OII3726_FLUX_ERR']]
print(len(tmp))
mask = np.full_like(nebulae_mask_KCWI,np.nan)
bad = np.full_like(nebulae_mask_KCWI,np.nan)

for row in tmp:
    diff = 100*np.abs((row['OII_KCWI']-row['OII3726_FLUX'])/row['OII_KCWI'])
    mask[nebulae_mask_KCWI==row['region_ID']]=diff
    if np.isnan(diff):
        bad[nebulae_mask_KCWI==row['region_ID']]=1

im = ax1.imshow(mask,alpha=0.6,vmin=0,vmax=100)
#ax1.imshow(bad,alpha=0.9,vmin=0,vmax=1,cmap=plt.cm.Reds)
ax1.axis('off')
ax1.set_title(r'[O\,\textsc{ii}] KCWI (S/N$\geq 1$)')
#ax.set(xlim=[xmin,xmax],ylim=[ymin,ymax])

ax2 = fig.add_subplot(122,projection=OII_KCWI.wcs)
norm=simple_norm(OII_KCWI.data,clip=False,percent=99)
ax2.imshow(OII_KCWI.data,norm=norm,cmap=plt.cm.gray_r)

tmp = OII_fluxes[OII_fluxes['OII3726_FLUX']<1*OII_fluxes['OII3726_FLUX_ERR']]
print(len(tmp))
mask = np.full_like(nebulae_mask_KCWI,np.nan)
bad = np.full_like(nebulae_mask_KCWI,np.nan)

for row in tmp:
    diff = 100*np.abs((row['OII_KCWI']-row['OII3726_FLUX'])/row['OII_KCWI'])
    mask[nebulae_mask_KCWI==row['region_ID']]=diff
    if np.isnan(diff):
        bad[nebulae_mask_KCWI==row['region_ID']]=1

im = ax2.imshow(mask,alpha=0.6,vmin=0,vmax=100)
#ax2.imshow(bad,alpha=0.9,vmin=0,vmax=1,cmap=plt.cm.Reds)
ax2.axis('off')
ax2.set_title(r'[O\,\textsc{ii}] KCWI (S/N$<1$)')
#ax.set(xlim=[xmin,xmax],ylim=[ymin,ymax])
        
plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.1)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.6])
fig.colorbar(im,cax=cbar_ax,label=r'difference in percent')
        
#fig.colorbar(im,label='difference in percent')
plt.savefig(basedir/'reports'/'KCWI_SITELLE_regions_diff.pdf',dpi=300)

plt.show()

reproject line maps instead of masks

In [ ]:
import pyneb as pn
EBV_NGC2835 = 0.089
EBV_NGC0628 = 0.062

rc_MW = pn.RedCorr(R_V=3.1,E_BV=EBV_NGC2835,law='CCM89 oD94')


with fits.open(basedir/'data'/'data_v2p1'/'Nebulae_Catalogue_v2p1_OII.fits') as hdul:
    OII_fluxes = Table(hdul[1].data)
OII_fluxes = OII_fluxes[OII_fluxes['gal_name']=='NGC2835']
OII_fluxes['outside'] = [np.any(np.isnan(OII_KCWI_reprojected[nebulae_mask.data==region_ID])) for region_ID in OII_fluxes['region_ID']]
OII_fluxes = OII_fluxes[~OII_fluxes['outside']]
OII_fluxes['OII_KCWI'] = np.array([np.nansum(OII_KCWI_reprojected[nebulae_mask.data==region_ID]) for region_ID in OII_fluxes['region_ID']]) 
OII_fluxes['OII_KCWI'] *= 1e4*rc_MW.getCorr(3726)

print(f'{len(OII_fluxes)} regions in KCWI FOV')

In [ ]:
fig,ax=plt.subplots(figsize=(4,4))

lim = [1e3,1e6]
ax.scatter(OII_fluxes['OII_KCWI']/2.2,OII_fluxes['OII3726_FLUX'])

ax.plot(lim,lim,color='black')
ax.set(xscale='log',xlim=lim,yscale='log',ylim=lim,
       ylabel=r'[O\,\textsc{ii}] (SITELLE)',xlabel=r'[O\,\textsc{ii}] (KCWI)')

plt.show()

### Measure flux in random apertures

In [ ]:
from photutils import SkyCircularAperture, aperture_photometry

x,y=np.random.randint(low=(0,0),high=OII_KCWI.data.shape,size=(50,2)).T
coordinates = SkyCoord.from_pixel(x,y,wcs=OII_KCWI.wcs)

apertures = SkyCircularAperture(coordinates,2*u.arcsec)
fluxes_SITELLE = aperture_photometry(OII_SITELLE,apertures)
fluxes_KCWI = aperture_photometry(OII_KCWI,apertures)

pixel_aperture = aperture.to_pixel(OII_SITELLE.wcs)

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))

lim = [1e-19,1e-13]
ax.scatter(4*fluxes_SITELLE['aperture_sum'],1e-16*fluxes_KCWI['aperture_sum'])

ax.plot(lim,lim,color='black')
ax.set(xscale='log',xlim=lim,yscale='log',ylim=lim)

plt.show()

In [ ]:
from astrotools.metallicity import ratio_from_electron_density


O2 = pn.Atom('O', 2)
S2 = pn.Atom('S', 2)

ratios = np.linspace(0.4,1.5)

density_O2 = O2.getTemDen(int_ratio=ratios, tem=1e4, wave1=3729, wave2=3727)
density_S2 = S2.getTemDen(int_ratio=ratios, tem=1e4, wave1=6717, wave2=6731)

fig,ax=plt.subplots(figsize=(8,8/1.618))


n_e = np.logspace(1,5,200)
ax.plot(n_e,ratio_from_electron_density(n_e,'SII'),label='[SII]')
ax.plot(n_e,ratio_from_electron_density(n_e,'OII'),label='[OII]')

ax.plot(density_O2,ratios,label='[OII] pyneb')
ax.plot(density_S2,ratios,label='[SII] pyneb')

ax.legend()
ax.set(xscale='log',xlim=[10,1e4])
plt.show()

## Old vs New fitting routine

here we compare the previous fit (fixed flux ratio of 1.4, no good prior for velocity) with the updated version.

In [ ]:
OII_old = Table(fits.getdata(basedir/'data'/'data_v2p1'/'Nebulae_Catalogue_v2p1_OII.fits'))
OII_old = OII_old[np.isin(OII_old['gal_name'],['NGC0628','NGC2835','NGC3351','NGC4535'])]
OII_new = Table(fits.getdata(basedir/'data'/'data_v2p4'/'Nebulae_Catalogue_v2p1_OII.fits'))

In [ ]:
np.sum(~np.isnan(OII_new['OII3726_FLUX']))
np.sum(~np.isnan(OII_old['OII3726_FLUX']))

In [ ]:
fig,ax=plt.subplots()

ax.hist(OII_old['OII3726_FLUX']/OII_old['OII3726_FLUX_ERR'],bins=np.arange(0,10,1),label='old',alpha=0.7)
ax.hist(OII_new['OII3726_FLUX']/OII_new['OII3726_FLUX_ERR'],bins=np.arange(0,10,1),label='new',alpha=0.7)
ax.legend()
ax.set(xlim=[0,10])

plt.show()

In [ ]:
fig,ax=plt.subplots()

tmp = OII_new[OII_new['OII3726_broadening']<100]
ax.hist(tmp['OII3726_FLUX']/tmp['OII3726_FLUX_ERR'],bins=np.arange(0,10,1),label='narrow',alpha=0.7)

tmp = OII_new[OII_new['OII3726_broadening']>100]
ax.hist(tmp['OII3726_FLUX']/tmp['OII3726_FLUX_ERR'],bins=np.arange(0,10,1),label='broad',alpha=0.7)
ax.legend()
ax.set(xlim=[0,10])

plt.show()



In [ ]:
OII_old = Table(fits.getdata(basedir/'data'/'data_v2p1'/'fluxes'/'NGC2835_OII_fluxes.fits'))
OII_new = Table(fits.getdata(basedir/'data'/'data_v2p2'/'fluxes'/'NGC2835_OII_fluxes.fits'))

In [ ]:
fig,ax=plt.subplots(figsize=(5,5))
lim=[1e3,1e7]
ax.scatter(OII_old['OII3726_FLUX'],OII_new['OII3726_FLUX'],c=OII_new['OII3726_broadening'])
ax.plot(lim,lim,color='black')
ax.set(xscale='log',xlim=lim,yscale='log',ylim=lim,
      )

plt.show()

In [ ]:
OII_new[OII_new['OII3726_broadening']>2000]